# Анализ базы данных книжного сервиса

Компания купила крупный сервис для чтения книг по подписке. Нужно проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.
 

#### Цель проекта:
Проаналзировать базу данных книжного сервиса

#### План анализа:
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определим издательство, которое выпустило наибольшее число книг (возьмем книги толще 50 страниц);
4. Определим автора с самой высокой средней оценкой книг (будем учитывать только книги с 50 и более оценками);
5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Загрузка и изучение данных:

Импортируем библиотеки и установим соединение с базой данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


Изучим первые 5 строк в каждой таблице.

In [2]:
query = '''SELECT * 
           FROM books
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query_1 = '''SELECT * 
           FROM authors
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_1), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query_2 = '''SELECT * 
           FROM publishers
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_2), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query_3 = '''SELECT * 
           FROM ratings
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_3), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
pd.options.display.max_colwidth = 400
query_3 = '''SELECT * 
           FROM reviews
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_3), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


Данные отсортированы по алфавиту или по возрастанию, столбцы названы корректно.

Можно переходить к исследовательскому анализу данных.

### Исследовательский анализ данных

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query_4 = '''SELECT COUNT(book_id) AS count_books
             FROM books
             WHERE publication_date > '2000-01-01' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_4), con = con)

,count_books
0,819


После 1 января 2000 года вышло 819 книг из 1000, имеющихся в базе.

Для каждой книги посчитаем количество обзоров и среднюю оценку.

Отсортируем результат сначала по уменьшению средней оценки, затем - по уменьшению количества обзоров.

In [8]:
query_5 = '''SELECT b.book_id,
                    ROUND(AVG(rating), 2) AS avg_rating,
                    COUNT(review_id) AS count_review
             FROM books AS b
             LEFT JOIN reviews AS rew ON b.book_id=rew.book_id
             LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
             GROUP BY b.book_id
             ORDER BY avg_rating DESC, count_review DESC'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_5), con = con)

,book_id,avg_rating,count_review
0,17,5.00,16
1,553,5.00,12
2,444,5.00,9
3,347,5.00,6
4,610,5.00,4
...,...,...,...
995,915,2.25,12
996,202,2.00,9
997,371,2.00,4
998,316,2.00,4


В данных есть довольно много книг с максимальной оценкой пользователей, но вот количество обзоров на них довольно сильно отличается.

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [9]:
query_6 = '''SELECT publisher,
                    count_book
             FROM publishers
             INNER JOIN (SELECT p.publisher_id,
                        COUNT(book_id) AS count_book
                        FROM publishers AS p
                        LEFT JOIN books AS b ON p.publisher_id=b.publisher_id
                        WHERE num_pages > 50
                        GROUP BY p.publisher_id
                        ORDER BY count_book DESC
                        LIMIT 1) AS first_publ
             ON publishers.publisher_id=first_publ.publisher_id'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_6), con = con)

,publisher,count_book
0,Penguin Books,42


Больше всего книг (а именно 42) выпустило британское издательство Penguin Books.

Определим автора с самой высокой средней оценкой книг, будем учитывать только книги с 50 и более оценками.

In [10]:
query_7 = '''WITH avg_rat AS (SELECT book_id,
                                     AVG(rating) AS avg_rating,
                                     COUNT(rating_id) AS count_rating
                              FROM ratings
                              GROUP BY book_id
                              HAVING COUNT(rating_id) >= 50)
             SELECT author,
                    author_avg_rat
             FROM authors
             INNER JOIN (SELECT author_id,
                                AVG(avg_rating) AS author_avg_rat
                         FROM books AS b
                         INNER JOIN avg_rat ON b.book_id=avg_rat.book_id
                         GROUP BY author_id
                         ORDER BY author_avg_rat DESC
                         LIMIT 1) AS group_author_rat 
             ON authors.author_id=group_author_rat.author_id'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_7), con = con)

,author,author_avg_rat
0,J.K. Rowling/Mary GrandPré,4.283844


Автором с самой высокой средней оценкой книг оказалась Джоан Роулинг! Книги о Гарри Поттере действительно очень популярны.

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query_8 = '''WITH count_rat AS (SELECT username,
                                       COUNT(rating_id) AS count_rating
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating_id) > 48)
             SELECT AVG(count_rev) AS avg_count_reviews
             FROM (SELECT r.username,
                          COUNT(review_id) AS count_rev
                   FROM reviews AS r
                   INNER JOIN count_rat
                   ON r.username=count_rat.username
                   GROUP BY r.username) AS count_rev'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_8), con = con)

,avg_count_reviews
0,24.0


В среднем 24 обзора написали пользователи, которые поставили больше 48 оценок книгам, т.е. примерно на каждую вторую оцененную книгу пользователи пишут обзор. Довольно высокая активность.

### Вывод

Резюмируя все результаты можно сформировать общий вывод:
1. После 1 января 2000 года вышло 819 книг из 1000, имеющихся в базе.
2. В данных есть довольно много книг с максимальной оценкой пользователей, но вот количество обзоров на них довольно сильно отличается.
3. Больше всего из имеющихся в базе книг (а именно 42) выпустило британское издательство Penguin Books.
4. Автором с самой высокой средней оценкой книг, имеющихся в базе данных, оказалась Джоан Роулинг.
5. В среднем 24 обзора написали пользователи, которые поставили больше 48 оценок книгам, т.е. примерно на каждую вторую оцененную книгу пользователи пишут обзор.